In [163]:
from gurobipy import Model, quicksum, GRB

In [177]:
Plnt = {'plnt'}
DC = set(range(1,3))
Cust = DC.copy()
TRoute = {(i,j) for i in Plnt for j in DC}
DRoute = {(j,k) for j in DC for k in Cust}
Route = T_Route | D_Route

T = 2
Period = list(range(1, T+1))

In [178]:
tp_cost = {(i,j):5 for i,j in TRoute}
del_cost = {(j, k):10 for j,k in DRoute}
demand = {(k,t):10 for k in Cust for t in Period}
run_dc_cost = {j:1000 for j in DC}

In [179]:
model = Model()

In [180]:
x,y = {}, {}

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for j in DC:
    for t in Period:
        y[j,t] = model.addVar(vtype="B", name=f'y[{j},{t}]')

model.update()

In [181]:
Cust_Demand_Cons, DC_Running_Cons, DC_Flow_Cons = {}, {}, {}

for k in Cust:
    for t in Period:
        Cust_Demand_Cons[k,t] = model.addConstr(
            quicksum(x[j,k,t] for j in DC)
            ==
            demand[k,t]
        )

for j,k in DRoute:
    for t in Period:
        DC_Running_Cons[j,k,t] = model.addConstr(
            x[j,k,t]
            <=
            demand[k,t] * y[j,t]
        )

for j in DC:
    for t in Period:
        DC_Flow_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in Plnt)
            ==
            quicksum(x[j,k,t] for k in Cust)
        )
    
model.update()

In [182]:
model.setObjective(
    quicksum(tp_cost[i,j] * x[i,j,t] for i,j in TRoute for t in Period) +
    quicksum(del_cost[j,k] * x[j,k,t] for j,k in DRoute for t in Period) +
    quicksum(run_dc_cost[j] * y[j,t] for j in DC for t in Period)
    ,GRB.MINIMIZE
)
model.update()

In [183]:
model.optimize()

Optimize a model with 16 rows, 28 columns and 36 nonzeros
Variable types: 24 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 16 rows and 28 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 2600 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.600000000000e+03, best bound 2.600000000000e+03, gap 0.0000%


In [184]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

x[2,1,1] 10.0
x[2,1,2] 10.0
x[plnt,2,1] 20.0
x[plnt,2,2] 20.0
x[2,2,1] 10.0
x[2,2,2] 10.0
y[2,1] 1.0
y[2,2] 1.0
